# Bag of words features

Data is in format:

ID,Gene,Variation,Class

1513,ALK,RANBP2-ALK Fusion,7

2222,PTEN,K128N,4

1672,FLT3,D835E,7

3254,CASP8,CASP8L,4

In [63]:
#compute bag of words tf-idf for text
#* One hot encode mutation class for outputs
#* One hot encode gene input
#* use bag of words tf-idf for variation input


In [93]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
import utils as U
reload(U)

<module 'utils' from 'utils.py'>

## Feature generation

In [94]:
train_variants = U.load_csv("data/train/train/training_variants.csv", delimiter="|")[1:]
train_text = U.load_csv("data/train/train/training_text.csv", delimiter="|")[1:]
valid_variants = U.load_csv("data/train/valid/valid_variants.csv", delimiter="|")[1:]
valid_text = U.load_csv("data/train/valid/valid_text.csv", delimiter="|")[1:]

id_class = U.id_class()

In [96]:
results = [[int(id_class[id])] for id in train_text[:, 0]]
val_results = [[int(id_class[id])] for id in valid_text[:, 0]]

In [97]:
encoder = OneHotEncoder()
results = encoder.fit_transform(results)
results = results.toarray()

val_results = encoder.fit_transform(val_results)
val_results = val_results.toarray()

In [105]:
vectorizer = TfidfVectorizer()
vectorizer.fit(np.concatenate([train_text[:, 1], valid_text[:, 1]]))
train_tfidf_values = vectorizer.transform(train_text[:, 1])
valid_tfidf_values = vectorizer.transform(valid_text[:, 1])

train_data = train_tfidf_values.toarray()
val_data =  valid_tfidf_values.toarray()
print(train_data.shape)
print(val_data.shape)

(2900, 139923)
(421, 139923)


## Model

In [106]:
from keras.layers import Dense, Input
from keras.optimizers import Adam
from keras.models import Model
inps = Input(shape=(139923,))
outs = Dense(9, activation="softmax")(inps)
model = Model(inputs=inps, outputs=outs)
model.compile(optimizer=Adam(lr=0.01), metrics=['accuracy'], loss="categorical_crossentropy")

In [108]:
model.fit(train_data, results, epochs=10, validation_data=(val_data, val_results))

Train on 2900 samples, validate on 421 samples
Epoch 1/10
2900/2900 [==============================] - 9s - loss: 0.9234 - acc: 0.7186 - val_loss: 1.0427 - val_acc: 0.6413